# Lecture6 Dependency Parsing

## 语言学的两种观点

如何描述语法，有两种主流观点，短语结构和依存结构。

### 短语结构

其中一种是短语结构文法，英文术语是：Constituency = phrase structure grammar = context-free grammars (CFGs)。

这种短语语法用固定数量的rule分解句子为短语和单词、分解短语为更短的短语或单词……一个取自WSJ语料库的短语结构树示例：

![6cbb8645gw1fb9et7tnw3j211i0k2n0e.jpg](https://i.loli.net/2018/07/19/5b50571c07a34.jpg)

### 依存结构

另一种是依存结构，用单词之间的依存关系来表达语法。如果一个单词修饰另一个单词，则称该单词依赖于另一个单词。

![屏幕快照 2018-07-19 下午4.38.58.png](https://i.loli.net/2018/07/19/5b50571c353f2.png)

### 歧义

通过句法树可以表达歧义，一个确定的句法树对应句子的一个确定解读，比如对介词短语依附(attachment of prepositional phrases (PPs)):

![006Fmjmcly1fgij616l29j311i0ps0vn.jpg](https://i.loli.net/2018/07/19/5b50571c2f0c0.jpg)

from space这个介词短语到底依附谁？不同的答案导致对句子不同的理解。

### 依附歧义

很难确定如何把一个短语（介词短语、状语短语、分词短语、不定式）依附到其他成分上去，比如下列句子：

![006Fmjmcly1fgijfj3n2vj31ds0fwter.jpg](https://i.loli.net/2018/07/19/5b50571c0b888.jpg)

每个括号中都是一个短语，它们依附的对象各不相同。对于$n$个短语来讲，组成的树形结构有$Cn=\frac{(2n)!}{(n+1)!n!}$。这是Catalan数，指数级增长，常用于树形结构的计数问题。


## 标注数据集的崛起：Universal Dependencies treebanks

虽然上下文无关文法中的语法集很容易写，无非是有限数量的规则而已，但人工费时费力标注的树库却茁壮成长了起来。在1993年首次面世的Universal Dependencies treebanks如今在Google的赞助下发布了2.0，其授权大多是署名-相同方式共享，覆盖了全世界绝大多数语言（不包括简体中文）。

其官网是：http://universaldependencies.org/

GitHub主页是：https://github.com/UniversalDependencies

树库示例：

![006Fmjmcly1fgikc4szn1j30tm0howhf.jpg](https://i.loli.net/2018/07/19/5b50571c3afdf.jpg)

人们偏好树库多于规则的原因是显而易见的，树库虽然标注难度高，但每一份劳动都可被复用（可以用于词性标注命名实体识别等等任务）；而每个人编写的规则都不同，并且死板又丑陋。树库的多用性还是得其作为评测的标杆数据，得到了越来越多的引用。


## 依存文法与依存结构

这节课以及练习用的都是依存句法树，而不是短语结构树。这并不是随机选择，而是由于前者的优势。90年代的句法分析论文99%都是短语结构树，但后来人们发现依存句法树标注简单，parser准确率高，所以后来（特别是最近十年）基本上就是依存句法树的天下了（至少80%）。

不标注依存弧label的依存句法树就是短语结构树的一种：

![006Fmjmcly1fgikxq72uqj30xy0kmacl.jpg](https://i.loli.net/2018/07/19/5b50571c30f9c.jpg)

一旦标上了，两者就彻底不同了：

![006Fmjmcly1fgikyihe3lj30yi0kuwhw.jpg](https://i.loli.net/2018/07/19/5b50571c38f39.jpg)

依赖语法假定语法结构由词汇项之间的关系组成，通常是称为依赖关系的二元不对称关系（“箭头”），这里箭头的尾部是head（被修饰的主题），箭头指向的是dependent（修饰语）。每个句子都有一个虚根，代表句子之外的开始，这样句子中的每个单词都有自己的依存对象了。

### 起源

> 语法依存的概念可以追溯到公元前4世纪印度语言学家Panini对语义、句法和形态依存的分类研究，但一般认为现代依存语法理论的创立者是法国语言学家Lucien Tesnière（1893—1954）。L.Tesnière的思想主要反映在他1959年出版的《结构句法基础》（Eléments de syntaxe structurale）一书中［Tesnière，1959］。
> ——《统计自然语言处理》

课件中还有更详细的历史，不太感兴趣，跳过；只记录一点，第一个computational句法分析器也是依存句法分析器。

### 句法分析可用的特征

- 双词汇亲和（Bilexical affinities），比如discussion与issues

- 词语间距，因为一般相邻的词语才具有依存关系

- 中间词语，如果中间词语是动词或标点，则两边的词语不太可能有依存

- 词语配价，一个词语最多有几个依赖者

### 依存句法分析

有几个约束条件：

- ROOT只能被一个词依赖

- 无环

英语中大部分句子是projective的，少数是non-projective的：

![006Fmjmcly1fgm6p51a0oj31eg070abc.jpg](https://i.loli.net/2018/07/19/5b50571b9ed2c.jpg)

有个学生问是否可以将一个依存句法树还原成句子，答案是否定的。

文献中的依存句法分析方法有：

- Dynamic programming

    Eisner（1996）给出了一个复杂度为O（n3）的算法，通过在末端而不是在中间生成head的解析项
    
- Graph algorithms

    最小生成树
    
- Constraint Satisfaction

    在某个图上逐步删除不符合要求的边，直到成为一棵树
    
- “Transition-based parsing” or “deterministic dependency parsing”

    基于贪心决策动作拼装句法树，已经证明非常有效
    
### Arc-standard transition

动作体系的formal描述如下：

![006Fmjmcly1fgncd2xnfmj319i0gytbl.jpg](https://i.loli.net/2018/07/19/5b505cb6a983b.jpg)

*以下图解参考自[Arc-standard详解](http://www.hankcs.com/nlp/parsing/neural-network-based-dependency-parser.html/2#h2-6)*

栈$s$是用来储存系统已经处理过的句法子树的根节点的，初始状态下$s=[ROOT]$。另外，定义从栈顶数起的第$i$个元素为$s_{i}$。那么栈顶元素就是$s_{1}$，$s_{1}$的下一个元素就是$s_{2}$：

![6cbb8645gw1exvwgovv17j202s08daa2.jpg](https://i.loli.net/2018/07/19/5b505cb6141d0.jpg)

在一些中文论文中习惯使用焦点词这个表述，我觉得该表述更形象，如果我们将栈横着放，亦即让先入栈的元素在左边，后入栈的元素在右边：

![6cbb8645gw1exvweou5nzj208e02tdfw.jpg](https://i.loli.net/2018/07/19/5b505cb613a29.jpg)

则称s2为左焦点词，s1为右焦点词。接下来的动作都是围绕着这两个焦点词展开的。

一条依存弧有两个信息：动作类型+依存关系名称l。l视依存句法语料库中使用了哪些依存关系label而定，在arc-standard系统中，一共有如下三种动作：

- LEFT-ARC(l)

    添加一条$s_{2}->s_{1}$的依存边，名称为$l$，并且将$s_{2}$从栈中删除。前提条件：$|S|\ge2$。亦即建立右焦点词依存于左焦点词的依存关系，例如：
    
    ![6cbb8645gw1exvxaehp9cj20s503imy1.jpg](https://i.loli.net/2018/07/19/5b505cb670621.jpg)
    
- RIGHT-ARC(l)

    添加一条$s_{2}->s_{1}$的依存边，名称为$l$，并且将$s_{1}$从栈中删除。前提条件：$|S|\ge2$。亦即建立左焦点词依存于右焦点词的依存关系，例如：
    
    ![6cbb8645gw1exvxex7469j20rk05hdgu.jpg](https://i.loli.net/2018/07/19/5b505cb671ed6.jpg)
    
- SHIFT

    将$b_{1}$出队，压入栈。亦即不建立依存关系，只转移句法分析的焦点，即新的左焦点词是原来的右焦点词，依此类推。例如：
    
    ![6cbb8645gw1exvxlutefwj20r603sjs9.jpg](https://i.loli.net/2018/07/19/5b505cb673717.jpg)
    
可见当存在$N_{l}$种依存关系时，系统一共有$|T|=2N_{l}+1$种变换。

### MaltParser

无搜索，贪婪地下转移决策，线性复杂度，只损失了一点效果。加个beam search会上升一点。

### 传统特征表示

![006Fmjmcly1fgnck7jp2xj30nj0eitc9.jpg](https://i.loli.net/2018/07/19/5b505cb6a0797.jpg)

无非是栈和队列中单词、词性、依存标签的组合的特征函数，一个超长的稀疏01向量。

### 效果评估

评测指标是UAS（不考虑标签只考虑弧）或LAS（同时考虑标签和弧）：

![006Fmjmcly1fgncnr38ruj31e00r20y2.jpg](https://i.loli.net/2018/07/19/5b505da50946f.jpg)

依存弧可以用来判断蛋白质的相互作用：

![006Fmjmcly1fgncqiidm8j31a80o2te3.jpg](https://i.loli.net/2018/07/19/5b505da50b781.jpg)

### 投射性

CFG转换得到的依存树一定是投射性的，但依存理论允许非投射性的依存句法树（一些语义需要通过非投射性表达）。

![006Fmjmcly1fgncuqs168j31co0cojud.jpg](https://i.loli.net/2018/07/19/5b505da35f728.jpg)

arc-standard算法只能拼装投射性的句法树，但换个体系、加上后处理、采用graph-based方法就能得到非投射的句法树。

## 为什么需要神经网络句法分析器

传统特征表示稀疏、不完全、计算代价大（SVM之类的线性分类器本身是很快的，而传统parser的95%时间都花在拼装查询特征上了）。

### 神经网络依存句法分析器

无非是传统方法拼接单词、词性、依存标签，新方法拼接它们的向量表示：

![006Fmjmcly1fgnda5d97hj310c0nk785.jpg](https://i.loli.net/2018/07/19/5b505ec383e46.jpg)

然后模型是一个司空见惯的三层神经网络：

![006Fmjmcly1fgndbkj631j31eu0qwgta.jpg](https://i.loli.net/2018/07/19/5b505ec3a38d5.jpg)

事实上，在“深度学习”“神经网络”的喧嚣中冷静下来回顾一下，相较于传统的graph-based方法，花了这么多功夫得到的只是0.1%的LAS提升：

![006Fmjmcly1fgnd5teln2j31aw0k0gon.jpg](https://i.loli.net/2018/07/19/5b505ec399e92.jpg)

### 为何需要非线性

有很多S形函数可选：

![006Fmjmcly1fgndj6nluuj313y0k00wh.jpg](https://i.loli.net/2018/07/19/5b505ec362722.jpg)

tanh是sigmod的缩放偏移版：

$$tanh(z)=2logistic(2z)−1$$

还有其他各种备选：

![006Fmjmcly1fgndl9t5q6j31cm0ksq8d.jpg](https://i.loli.net/2018/07/19/5b505ec3a0843.jpg)

其中ReLU成为新贵，Manning说他简直不敢相信这种疯狂的函数竟然效果最好（也许神经网络本来就是疯狂的）。ReLU在反向传播时要么不反馈残差，要么原样反馈残差。

### 未来工作

Chen&Manning的工作被许多人继续往前推进，走在最前沿的是Google。趋势是：

更大更深调参调得更好（更昂贵）的神经网络

Beam Search

在决策序列全局进行类似CRF推断的方法（CRF宝刀未老，老当益壮啊）

Google的SyntaxNet 中的 Parsey McParseFace的效果：

![006Fmjmcly1fgndqwd3yaj315e0asmz0.jpg](https://i.loli.net/2018/07/19/5b505ec377000.jpg)